### This script is taking all the dedication text, processing it, and then calling paraphrasing model to perform text based Augmentation. Then save the result as a image(.png) format. Finally, it goes through all the images and perform image transformation and save the result as .png format.

In [1]:
## importing the modules from ETDAugmentation script
from ipynb.fs.full.ETDaugmentation import *
from imgaug import augmenters as iaa
import imgaug as ia
import pandas as pd
import numpy as np
import cv2
import glob
import re
import time
from tqdm import tqdm
import tensorflow as tf
import torch
import textwrap
from transformers import PegasusForConditionalGeneration, PegasusTokenizerFast, PegasusTokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

2023-05-04 22:01:09.251564: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-04 22:01:09.527124: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


2023-05-04 22:01:42.709767: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-04 22:01:43.514662: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:0 with 6837 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:1d:00.0, compute capability: 7.0


In [3]:
## loading the paraphrasing model

#references: https://arxiv.org/abs/1912.08777

model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

In [6]:
class aug:
    
    def get_paraphrased_sentences(self, input_text, num_return_sequences):
        self.input_text = input_text
        self.num_return_sequences = num_return_sequences        
        batch = tokenizer([self.input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
        translated = model.generate(**batch,max_length=60,num_beams=10, num_return_sequences=self.num_return_sequences, temperature=1.5)
        paraphrased_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
        return paraphrased_text
    
    def paraphrased_text(self, class_text_list):
        self.class_text_list = class_text_list
        phrases_text = []
        final_phrases = []
        for ele in self.class_text_list:
            phrase = self.get_paraphrased_sentences(ele, 4)
            for idx in phrase:
                phrases_text.append([idx])       

            for txt in phrases_text:
                para_phrases = [''.join(x) for x in txt]
                paraphrase_ = [''.join(x for x in para_phrases)] ## combine  the splitted lists into a paragraph
                paraphrase_text = str(paraphrase_).strip('[]').strip("'")
                paraphrase_strip = paraphrase_text.strip('""')
                final_phrases.append(paraphrase_strip)
                
        return final_phrases
    
    def wrap_text(self, text):
        self.text = text
        new_phrase = textwrap.wrap(self.text, width=90)
        string = ''
        for ele in new_phrase[0:]:
            string = string + ele + '\n'
        return string

In [7]:
class DEDICATION_TEXT_IMG_AUG:
    
    def list_of_phrases(self, phrases_dedication):
        phrases_list = []
        self.phrases_dedication = phrases_dedication
        for text in tqdm(self.phrases_dedication, desc = 'Processing list of phrases'):
            dedication = parser.preprocess_dedication(text)
            dedication_paraphrase = augmentation.paraphrased_text(dedication)
            dedication_list = [dedication_paraphrase]
            phrases_list.append(dedication_list)
        
        return phrases_list

    ## This function is taking all the augmented text and wrapping it so that it can fit into the page
    ## it also adds heading to the top of the page

    def list_of_dedication(self, dedication_text):
        list_dedication = []
        self.dedication_text = dedication_text
        for text in self.dedication_text:
            for row in text:
                for idx in tqdm(row, desc = 'Processing list of dedication'):
                    dedication_text_wrap = augmentation.wrap_text(idx)
                    dedication_title = "\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tDedication\n" + dedication_text_wrap
                    list_dedication.append(dedication_title)
            
        return list_dedication
    
    def text_on_img(self, fileName, text, size):
        self.fileName = fileName
        self.text = text
        self.size = size
        self.W = 2360
        self.H = 3200
        font = ImageFont.truetype('NimbusRoman-Regular.otf', size)
        image = Image.new(mode = "RGB", size = (self.W, self.H), color = "white")
        draw = ImageDraw.Draw(image)
        w, h = draw.textsize(text)
        ## Adjust the texual position in a page and draw the text on a image
        '''
        dedication -- 14 width and height 8; 
        ack -- 15 width and 6 height; 
        gabs -- 13 width and height 16
        abstract -- 16 width and height 12
        chapter abstract -- 18 width and height 6
        '''
        draw.text(((self.W-w)/14,(self.H-h)/8), text, font=font, fill=(0,0,0), spacing=60)
        image.save(fileName)
        
        return image
    
    ## This function is calling text_on_image function and save the image to the directory
    def dedication_aug(self, dedication_text_title):
        self.dedication_text_title = dedication_text_title
        for n, row in tqdm(enumerate(self.dedication_text_title), desc = 'Processing augmented images'):
            fileName = ("/home/mchou001/Label-Dedication/aug_text{}.png".format(n))
            save_image = self.text_on_img(fileName, row, 43) ## dedication -- 43, ack and abstract - 40, gabs -- 38
        
        return save_image
    
    def img_transform(self, img_files):
        self.img_files = img_files
        
        img_augmentation = iaa.Sequential([
            #iaa.Affine(rotate=(-6, 6)),
            #iaa.AdditiveGaussianNoise(scale=(10, 50)),
            iaa.SaltAndPepper(p=0.01),
            #iaa.GaussianBlur(sigma=0.5),
            #iaa.LinearContrast(alpha=1),
            #iaa.PerspectiveTransform(scale=0.025, keep_size=True)
        ], random_order = True)
        
        for n, images in tqdm(enumerate(self.img_files[0:]), desc = 'Processing augmented text to images'):
            aug_image = img_augmentation(image=cv2.imread(images))
            cv2.imwrite('./Label-Dedication/502/aug{}.png'.format(n),aug_image)
        
        # lower = 0
        # upper = 1000
        # while lower < upper:
        #     for n, images in tqdm(enumerate(self.img_files[0:]), desc = 'Processing augmented text images'):
        #         aug_image = img_augmentation(image=cv2.imread(images))
        #         cv2.imwrite('./Label-Dedication/502/aug{}.png'.format(lower),aug_image)
        #         lower += 1

In [8]:
if __name__ == "__main__":
    
    start = time.time()
    with tf.device('/gpu:0'):
    
        parser = Preprocessor()
        augmentation = aug()

        dedication_text = DEDICATION_TEXT_IMG_AUG()

        etd_data = pd.read_csv('ETD_aug.csv')
        etd_data.set_index("class", inplace = True)

        label_dedication = etd_data.loc["Label-Dedication"]
        phrases_dedication = label_dedication['text']

        dedication = dedication_text.list_of_phrases(phrases_dedication)
        
        dedication_ = dedication_text.list_of_dedication(dedication)
        
        df = pd.DataFrame(dedication_, columns = ['text'])
        df.to_csv('./Label-Dedication/dedication.csv', encoding = 'utf-8', index= None)

        dedication_text_aug = dedication_text.dedication_aug(dedication_)

        img_path = sorted(glob.glob('./Label-Dedication/*.png'), key = parser.numericalSort)

        augmented_img = dedication_text.img_transform(img_path)
    
    stop = time.time()
    print(f'Generating Images took: {(stop-start)/60} minutes')

2023-05-04 22:02:29.736519: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6837 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:1d:00.0, compute capability: 7.0
Processing list of dedication: 100%|██████████| 12/12 [00:00<00:00, 23663.21it/s]
Processing augmented images: 992it [05:34,  2.96it/s]
Processing augmented text to images: 992it [07:50,  2.11it/s]

Generating Images took: 14.43352021376292 minutes


In [9]:
import os

# folder path
dir_path = r'./Label-Dedication/502/'
count = 0
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('File count:', count)

File count: 992


In [3]:
#!zip -r dedication_txt_img.zip ./Label-Dedication/502/

In [10]:
parser = Preprocessor()
image_path = sorted(glob.glob('/home/mchou001/Label-Dedication/502/*.png'), key = parser.numericalSort)

df = pd.DataFrame(image_path, columns = ['img_path'])
df.to_csv('./Label-Dedication/dedication_img_aug.csv', encoding = 'utf-8', index= None)

In [12]:
df1 = pd.read_csv('./Label-Dedication/dedication_img_aug.csv')
df2 = pd.read_csv('./Label-Dedication/dedication.csv')
df1

,img_path
0,/home/mchou001/Label-Dedication/502/aug0.png
1,/home/mchou001/Label-Dedication/502/aug1.png
2,/home/mchou001/Label-Dedication/502/aug2.png
3,/home/mchou001/Label-Dedication/502/aug3.png
4,/home/mchou001/Label-Dedication/502/aug4.png
...,...
987,/home/mchou001/Label-Dedication/502/aug987.png
988,/home/mchou001/Label-Dedication/502/aug988.png
989,/home/mchou001/Label-Dedication/502/aug989.png
990,/home/mchou001/Label-Dedication/502/aug990.png


In [13]:
df3 = [txt.strip('\t') for txt in df2['text']]
df4 = pd.DataFrame(df3, columns = ['text'])
df4['class'] = np.nan
df4 = df4.fillna("Label-Dedication")
df4.to_csv('./Label-Dedication/updated_dedication.csv', encoding = 'utf-8', index= None)
df4

,text,class
0,Dedication\nIt was dedicated to my parents and...,Label-Dedication
1,Dedication\nDedicated to my family.\n,Label-Dedication
2,Dedication\nIt is dedicated to my parents and ...,Label-Dedication
3,Dedication\nIt was dedicated to my family.\n,Label-Dedication
4,Dedication\n4.\n,Label-Dedication
...,...,...
987,Dedication\nThe song was reproduced with the p...,Label-Dedication
988,Dedication\nPermission is required for further...,Label-Dedication
989,Dedication\nPermission is needed for further r...,Label-Dedication
990,Dedication\nPermission is required for any fur...,Label-Dedication


In [14]:
df5 = pd.read_csv('./Label-Dedication/updated_dedication.csv')
df6 = pd.concat([df1, df4], axis = 1)
df6.to_csv('./Label-Dedication/ded.csv', encoding = 'utf-8', index= None)

In [15]:
df6

,img_path,text,class
0,/home/mchou001/Label-Dedication/502/aug0.png,Dedication\nIt was dedicated to my parents and...,Label-Dedication
1,/home/mchou001/Label-Dedication/502/aug1.png,Dedication\nDedicated to my family.\n,Label-Dedication
2,/home/mchou001/Label-Dedication/502/aug2.png,Dedication\nIt is dedicated to my parents and ...,Label-Dedication
3,/home/mchou001/Label-Dedication/502/aug3.png,Dedication\nIt was dedicated to my family.\n,Label-Dedication
4,/home/mchou001/Label-Dedication/502/aug4.png,Dedication\n4.\n,Label-Dedication
...,...,...,...
987,/home/mchou001/Label-Dedication/502/aug987.png,Dedication\nThe song was reproduced with the p...,Label-Dedication
988,/home/mchou001/Label-Dedication/502/aug988.png,Dedication\nPermission is required for further...,Label-Dedication
989,/home/mchou001/Label-Dedication/502/aug989.png,Dedication\nPermission is needed for further r...,Label-Dedication
990,/home/mchou001/Label-Dedication/502/aug990.png,Dedication\nPermission is required for any fur...,Label-Dedication
